In [1]:
import geopandas as gpd
import pandas as pd
from siuba import *

# project sources

## dropped/excluded

* US 50 (withdrawn)
* US 101 Marin (minimal accessibility/speed benefits, worthwhile project better analyzed in another context)
* MST reroute (apparent marginal/negative benefits, institutional barriers to relocating/adding service)

# read and aggregate

In [2]:
first_set = gpd.read_file('./june16_corridor_metrics_speeds.geojson')
first_set = first_set >> filter(-_.corridor.str.contains('us50')) #dropped
first_set['submission_round'] = 1

In [3]:
second_set = gpd.read_file('./sep9_corridor_metrics_speeds.geojson')
second_set['submission_round'] = 2

In [14]:
access_results = pd.read_csv('./AccessibilityResults.csv')

In [15]:
first_set >> head(3)

,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,corridor,geometry,submission_round
0,28,28,City and County of San Francisco,9.3,11.8,13.2,480.1,1690.8,56.4,159,14.9,5.3,8.5,5.6,1.1,1739.0,16,us101_lombard,"POLYGON ((-214489.679 -21257.665, -214484.673 ...",1
1,43,43,City and County of San Francisco,8.1,9.5,10.1,1210.7,1690.8,73.2,143,14.8,4.8,16.5,5.4,1.1,NaN,16,us101_lombard,"POLYGON ((-214489.679 -21257.665, -214484.673 ...",1
2,28,28,City and County of San Francisco,9.6,11.5,11.9,1061.7,1347.9,56.4,159,14.9,5.3,18.8,6.0,2.4,3850.5,16,sr1_19th_a,"POLYGON ((-217949.193 -26918.889, -217955.214 ...",1


In [16]:
second_set >> head(3)

,route_id,route_short_name,organization,p20_corr_mph,p50_corr_mph,avg_corr_mph,speed_delay_minutes,total_speed_delay,p50_runtime_minutes,n_trips,span_hours,daily_avg_trips_hr,trips_added,new_avg_trips_hr,length_miles,distance_meters,target_mph,corridor,geometry,submission_round
0,4-13168,4,Los Angeles County Metropolitan Transportation...,4.7,11.1,9.3,94.2,94.2,96.2,33,2.3,7.2,1.0,7.4,1.2,1982.4,16,sr2_am,"POLYGON ((140814.209 -441265.975, 140892.548 -...",2
1,4-13168,4,Los Angeles County Metropolitan Transportation...,4.9,6.8,7.9,342.4,342.4,153.5,42,3.1,6.7,2.2,7.1,1.3,2105.6,16,sr2_pm,"POLYGON ((140814.209 -441265.975, 140892.548 -...",2
2,4-13168,4,Los Angeles County Metropolitan Transportation...,3.4,8.8,7.8,685.4,685.4,131.2,111,6.9,8.0,5.2,8.4,1.3,2105.6,16,sr2_mid,"POLYGON ((140814.209 -441265.975, 140892.548 -...",2


In [17]:
all_corridors = pd.concat([first_set, second_set])

In [18]:
all_grouped = (all_corridors >> group_by(_.corridor, _.organization, _.submission_round)  # length and span cause fan-out, re-add later...
               >> summarize(sum_trips =_.n_trips.sum(), total_delay = _.total_speed_delay.max())
               >> mutate(avg_delay_minutes = _.total_delay / _.sum_trips)
)

## note: don't sum/aggregate accessibility results within a corridor

In [20]:
rider_access_joined = all_grouped >> left_join(_, access_results, on=['corridor', 'submission_round'])

# Adding ridership (added to accessibility csv)

## apportioning ridership for partial-hour bus lane on sr2

* evenly during daytime hours (0500 - 2200), match post-covid trends of less peaking
* 22,669 daily for entire corr
* 

## notes
* for SDMTS, sum ridership from routes 60, 235, 225

In [11]:
metro_4_ridership = 22669
riders_per_hour = metro_4_ridership / (22 - 5)

In [12]:
second_set >> filter(_.corridor.str.contains('sr2')) >> select(_.corridor, _.span_hours) >> mutate(ridership = _.span_hours * riders_per_hour)

,corridor,span_hours,ridership
0,sr2_am,2.3,3066.982353
1,sr2_pm,3.1,4133.758824
2,sr2_mid,6.9,9200.947059


# Testing metrics

In [22]:
rider_access_joined >> head(3)

,corridor,organization,submission_round,sum_trips,total_delay,avg_delay_minutes,avg_pct_chng_access,pct_change_access_Jobs,pct_change_access_weighted_Jobs,avg_change_access_Jobs,weighted_avg_change_access_Jobs,pct_change_access_POIs,pct_change_access_weighted_POIs,avg_change_access_POIs,weighted_avg_change_access_POIs,daily_riders,pct_li_sub50k
0,i805,San Diego Metropolitan Transit System,2,90,99.5,1.105556,0.015327,0.015211,0.017178,216.299637,1308.461745,0.012527,0.013475,2.518450,14.145989,4233.0,0.79
1,sr123,Alameda-Contra Costa Transit District,2,219,872.0,3.981735,0.001397,0.001457,0.001271,75.015147,338.098215,0.001654,0.001523,0.953931,4.389842,14789.0,0.65
2,sr123_san_pablo,Alameda-Contra Costa Transit District,1,250,3623.2,14.492800,0.003879,0.003989,0.003270,209.573414,905.891038,0.005166,0.004487,3.029337,13.415441,14789.0,0.65


Create defined projects from disaggregated sr2

In [24]:
df = rider_access_joined >> filter(_.corridor.str.contains('sr2'), _.submission_round == 2)

In [32]:
df

,corridor,organization,submission_round,sum_trips,total_delay,avg_delay_minutes,avg_pct_chng_access,pct_change_access_Jobs,pct_change_access_weighted_Jobs,avg_change_access_Jobs,weighted_avg_change_access_Jobs,pct_change_access_POIs,pct_change_access_weighted_POIs,avg_change_access_POIs,weighted_avg_change_access_POIs,daily_riders,pct_li_sub50k
7,sr2_am,Los Angeles County Metropolitan Transportation...,2,33,94.2,2.854545,0.00165,0.001626,0.0017,195.925474,586.809451,0.001734,0.0016,2.475954,6.635036,3066.0,0.83
8,sr2_mid,Los Angeles County Metropolitan Transportation...,2,111,685.4,6.174775,0.00165,0.001626,0.0017,195.925474,586.809451,0.001734,0.0016,2.475954,6.635036,9200.0,0.83
9,sr2_pm,Los Angeles County Metropolitan Transportation...,2,42,342.4,8.152381,0.00165,0.001626,0.0017,195.925474,586.809451,0.001734,0.0016,2.475954,6.635036,4133.0,0.83


In [31]:
df2 = pd.DataFrame(df[['sum_trips', 'total_delay', 'daily_riders']].sum()).transpose()
df2['avg_delay_minutes'] = df2.total_delay / df2.sum_trips
df2['corridor'] = ['sr2_all_day']
df2['submission_round'] = 2
df2

,sum_trips,total_delay,daily_riders,avg_delay_minutes,corridor,submission_round
0,186.0,1122.0,16399.0,6.032258,sr2_all_day,2
